<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import os

#Load model

Tải pretrained model cho Tiếng Việt

In [ ]:
!pip install fasttext


  Using cached https://files.pythonhosted.org/packages/f8/85/e2b368ab6d3528827b147fdb814f8189acc981a4bc2f99ab894650e05c40/fasttext-0.9.2.tar.gz
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3096966 sha256=e34062d73b0465b928102841107a3593fbbf0056f9e8611f7eafe78cfb2127f7
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext.util


In [ ]:
fasttext.util.download_model('vi',if_exists='ignore')

Load

In [ ]:
ft = fasttext.load_model('/content/drive/MyDrive/DeepLearning_Application/Lab3-Word2Vec-With-Pretrained-Model/cc.vi.300.bin')

In [4]:
%cd /content/drive/MyDrive/DeepLearning_Application/Assignment3-SentimentAnalysis-with-LSTM/

/content/drive/MyDrive/DeepLearning_Application/Assignment3-SentimentAnalysis-with-LSTM


#Load tập từ vựng và ma trận word-embedding

In [5]:
currentDir = '/content/drive/MyDrive/DeepLearning_Application/Assignment3-SentimentAnalysis-with-LSTM/'
wordsList = np.load(os.path.join(currentDir, 'wordsList.npy'))
print('Simplified vocabulary loaded!')
wordsList = wordsList.tolist()

wordVectors = np.load(os.path.join(currentDir, 'wordVectors.npy'))
wordVectors = np.float32(wordVectors)
print ('Word embedding matrix loaded!')

Simplified vocabulary loaded!
Word embedding matrix loaded!


In [5]:
print('Size of the vocabulary: ', len(wordsList))
print('Size of the word embedding matrix: ', wordVectors.shape)

Size of the vocabulary:  19899
Size of the word embedding matrix:  (19899, 300)


In [6]:
import tensorflow as tf
maxSeqLength = 10   #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector
sentenceIndexes = np.zeros((maxSeqLength), dtype='int32')

# TODO 3.1: Gán chỉ số của các từ trong câu và 'sentenceIndexes'
sentence = 'Món này ăn hoài không hề biết chán'
words = sentence.split(" ")
for index,word in enumerate(words):
    word_idx = wordsList.index(word.lower())
    sentenceIndexes[index]=word_idx
print(sentenceIndexes)
#  Ma trận biểu diễn:
print('Sentence representation of word vectors:')
print(tf.nn.embedding_lookup(wordVectors,sentenceIndexes))

[  119  8136  4884 18791 16614 12231 15951  3371     0     0]
Sentence representation of word vectors:
tf.Tensor(
[[-0.1823 -0.0638  0.2376 ...  0.1462 -0.1092  0.0137]
 [ 0.027  -0.0542  0.1437 ... -0.0913  0.0114  0.0132]
 [ 0.021   0.0102  0.0096 ...  0.411  -0.2519  0.0151]
 ...
 [-0.0239 -0.0383  0.1734 ... -0.0677 -0.096   0.0045]
 [ 0.1882 -0.292   0.0072 ...  0.5919 -0.3094 -0.1228]
 [ 0.1882 -0.292   0.0072 ...  0.5919 -0.3094 -0.1228]], shape=(10, 300), dtype=float32)


Visual Dataset

In [ ]:
from os import listdir
from os.path import isfile, join
positiveFiles = ['positiveReviews/' + f for f in listdir('positiveReviews/') if isfile(join('positiveReviews/', f))]
negativeFiles = ['negativeReviews/' + f for f in listdir('negativeReviews/') if isfile(join('negativeReviews/', f))]
numWords = []
for pf in positiveFiles:
    with open(pf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)       
print('Positive files finished')

for nf in negativeFiles:
    with open(nf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)  
print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))

#Chuẩn hóa văn bản và tách từ

In [ ]:
import re
strip_special_chars = re.compile("[^\w0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [7]:
maxSeqLength = 180

Chạy và lưu vào file idsMatrix.npy

In [ ]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
nFiles = 0
# Index of Unknow word
unk_idx = wordsList.index('UNK')

for pf in positiveFiles:
    with open(pf, "r", encoding="utf-8") as f:
        nIndexes = 0
        line=f.readline()
        cleanedLine = cleanSentences(line)
        split = cleanedLine.split()
        for word in split:
            # TODO 3.2: Nếu 'word' thuộc tập 'wordsList' thì gán chỉ số của 'word' vào ma trận ids
            if word in wordsList:
               word_idx = wordsList.index(word)
               ids[nFiles][word_idx] = word_idx
            # Ngược lại: gán 'unk_idx' vào ma trận ids
            nIndexes = nIndexes + 1
            if nIndexes >= maxSeqLength:
                break
        nFiles = nFiles + 1 

print('Positive files are indexed!')
for nf in negativeFiles:
    with open(nf, "r", encoding="utf-8") as f:
        nIndexes = 0
        line=f.readline()
        cleanedLine = cleanSentences(line)
        split = cleanedLine.split()
        for word in split:
            # ToDo 3.2: tương tự như trên. Không khác gì hết.
            if word in wordsList:
               word_idx = wordsList.index(word)
               ids[nFiles][word_idx] = word_idx
            nIndexes = nIndexes + 1
            if nIndexes >= maxSeqLength:
                break
        nFiles = nFiles + 1 

print('Negative files are indexed!')
# Save ids Matrix for future uses.
np.save(os.path.join(currentDir,'idsMatrix.npy'), ids)

Load file đã chạy để tiết kiệm thời gian

In [8]:
ids = np.load(os.path.join(currentDir,'idsMatrix.npy'))
print('Word indexes of the first review: ', ids[0])


Word indexes of the first review:  [19898  1906  4454  5284 10661 11694 11994 18784 18569 18619 13174  9821
 14794  8884  6443  5767  8589 18850 15570  5596   799 11060  4222 16893
 13078  8136  3364  4454  4756 10304  8885  3553  9782  1232 14359 10606
   579 15522  2219 15092 14855 15253  4884  3364  5519  4558  9649   269
 15522 12309 14855 11503  2212  4884  7155 11577  4222  5767 15076 12225
 10774  1218  2876 19584  4558  2974 13452  5013   842 10642 17292 11895
   803 11060 16760  1906 15253 14598 15253  1047  5668  4884 10642 12225
  7090 17292 18109 13078 16334  1238  3364  5519  4135  3553 14967  4964
 15385  9673  2997 14855  7446  8038 11440  1345   842  5767   803 11060
 18791  5013     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0    

#Xây dựng hàm lấy dữ liệu train và test theo từng batch

In [15]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 2 == 0): 
            # Pick positive samples randomly
            num = randint(1,13999)
            labels.append([1,0])
        else:
            # Pick negative samples randomly
            num = randint(15999,29999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(13999,15999)
        if (num <= 14999):
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

#3.Xây dựng RNN Model với Tensorflow

In [9]:
# Initialize paramters
numDimensions = 300
batchSize = 64
lstmUnits = 128
nLayers = 2
numClasses = 2
iterations = 30000

In [10]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split

X = ids.copy()
Y = np.concatenate((np.ones((15000,1)),np.zeros((15000,1))))
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.067)

In [18]:
# Create a callback that saves the model's weights
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = keras.Sequential([
        layers.Embedding(input_dim=19899,output_dim=numDimensions,weights=[wordVectors],input_length=maxSeqLength,trainable=False),
        layers.LSTM(lstmUnits,return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(lstmUnits),
        layers.Flatten(),
        layers.Dense(2,activation='softmax')]
 )
model.save_weights(checkpoint_path.format(epoch=0))

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='Adam',
    metrics=["accuracy"])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 180, 300)          5969700   
_________________________________________________________________
lstm (LSTM)                  (None, 180, 128)          219648    
_________________________________________________________________
dropout (Dropout)            (None, 180, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 6,321,190
Trainable params: 351,490
Non-trainable params: 5,969,700
________________________________________

In [19]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batchSize, epochs=30000,callbacks=[cp_callback])

Epoch 1/30000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


438/438 [==============================] - 19s 37ms/step - loss: 0.6938 - accuracy: 0.5064 - val_loss: 0.6901 - val_accuracy: 0.5002

Epoch 00001: saving model to training_2/cp-0001.ckpt
Epoch 2/30000
438/438 [==============================] - 16s 36ms/step - loss: 0.6919 - accuracy: 0.5074 - val_loss: 0.6898 - val_accuracy: 0.5132

Epoch 00002: saving model to training_2/cp-0002.ckpt
Epoch 3/30000
438/438 [==============================] - 16s 36ms/step - loss: 0.6921 - accuracy: 0.5031 - val_loss: 0.6903 - val_accuracy: 0.5062

Epoch 00003: saving model to training_2/cp-0003.ckpt
Epoch 4/30000
438/438 [==============================] - 16s 36ms/step - loss: 0.6919 - accuracy: 0.5054 - val_loss: 0.6894 - val_accuracy: 0.5067

Epoch 00004: saving model to training_2/cp-0004.ckpt
Epoch 5/30000
438/438 [==============================] - 16s 36ms/step - loss: 0.6917 - accuracy: 0.5125 - val_loss: 0.6897 - val_accuracy: 0.5132

Epoch 00005: saving model to training_2/cp-0005.ckpt
Epoch 6/3

KeyboardInterrupt: ignored